In [1]:
# Word Prediction with Tensorflow RNN
# written by Sung Kyu Lim
# limsk@ece.gatech.edu
# 1/7/2019


# import tensorflow and numpy
import tensorflow as tf
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'


# array for alphabet
char_arr = [c for c in 'abcdefghijklmnopqrstuvwxyz']
print('char_arr:')
print(char_arr)


# assign array index to each alphabet
# ex: 'a': 0, 'b': 1, 'c': 2  ...
num_dic = {n: i for i, n in enumerate(char_arr)}
print('\nnum_dic:')
print(num_dic)


# num_dic testing 
seq = 'word'
input = [num_dic[n] for n in seq[0:-1]]
print('\nnum_dic encoding of "word":')
print(input)
target = num_dic[seq[3]]
print(target)


# batch testing
input_batch = []
input_batch.append(np.eye(26)[input])
print('\ninput_batch')
print(input_batch)

target_batch = []
target_batch.append(target)
print('\ntarget_batch')
print(target_batch)

/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


char_arr:
['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

num_dic:
{'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7, 'i': 8, 'j': 9, 'k': 10, 'l': 11, 'm': 12, 'n': 13, 'o': 14, 'p': 15, 'q': 16, 'r': 17, 's': 18, 't': 19, 'u': 20, 'v': 21, 'w': 22, 'x': 23, 'y': 24, 'z': 25}

num_dic encoding of "word":
[22, 14, 17]
3

input_batch
[array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]])]

target_batch
[3]


In [1]:
# Word Prediction with Tensorflow RNN
# written by Sung Kyu Lim
# limsk@ece.gatech.edu
# 1/7/2019


# import tensorflow and numpy
import tensorflow as tf
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'


# array for alphabet
char_arr = [c for c in 'abcdefghijklmnopqrstuvwxyz']


# assign array index to each alphabet
# ex: 'a': 0, 'b': 1, 'c': 2  ...
num_dic = {n: i for i, n in enumerate(char_arr)}


# training words and constants
seq_data = ['eager', 'enemy', 'fruit', 'grace', 'laugh', 'money', 'peace', 'party', 'ready', 'stage']
n_input = n_class = 26
n_stage = 4


# input encoder
def make_batch(seq_data):
    input_batch = []
    target_batch = []

    for seq in seq_data:
        input = [num_dic[n] for n in seq[0:n_stage]]
        target = num_dic[seq[-1]]
        input_batch.append(np.eye(26)[input])
        target_batch.append(target)
    return input_batch, target_batch


# global parameters
learning_rate = 0.01
n_hidden = 128
total_epoch = 100


# placeholders and variables
# the input placeolder should be 3-dimensional 
# in order to use tensorflow RNN cells
# note that Y, output label, is 1-dimensional
X = tf.placeholder(tf.float32, [None, n_stage, n_input])
Y = tf.placeholder(tf.int32, [None])
W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))


# two RNN cells and their deep RNN network
# we use dropout in cell 1
cell1 = tf.nn.rnn_cell.LSTMCell(n_hidden)
cell1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob=0.5)
cell2 = tf.nn.rnn_cell.LSTMCell(n_hidden)
cell2 = tf.nn.rnn_cell.DropoutWrapper(cell2, output_keep_prob=0.5)
cell3 = tf.nn.rnn_cell.LSTMCell(n_hidden)
cell3 = tf.nn.rnn_cell.DropoutWrapper(cell3, output_keep_prob=0.5)
cell4 = tf.nn.rnn_cell.LSTMCell(n_hidden)

multi_cell = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2, cell3, cell4])
outputs, states = tf.nn.dynamic_rnn(multi_cell, X, dtype=tf.float32)


# RNN output re-ordering and trimming in 3 steps
# [batch_size, n_stage, n_hidden] ->
# [n_stage, batch_size, n_hidden] ->
# [batch_size, n_hidden] 
# this cases the last stage output to be used
# model produces 26 floating point values
# prediction uses argmax to find which index model computes
outputs = tf.transpose(outputs, [1, 0, 2])
print(outputs.shape)
outputs = outputs[-1]
print(outputs.shape)
# outputs = outputs[]
# tmp = tf.reshape(outputs, [-1, 1, 2])
# print(tf.reshape(outputs, [-1, 1, 2]))
# print(tmp.shape)
model = tf.matmul(outputs, W) + b
# prediction = tf.cast(tf.argmax(model, 1), tf.int32)
prediction = tf.cast(tf.argmax(model, 1), tf.int32)
              

# prints model and prediction information
def info():
    sample = ['eager']
    input_batch, target_batch = make_batch(sample)
    m_info, p_info = sess.run([model, prediction], feed_dict={X: input_batch, Y: target_batch})

    print('word:', sample)
    print('model:', m_info)
    print('prediction:', p_info)
    print('predicted character', char_arr[p_info[0]])
    print()


# loss function and optimizer
# this new tool better handles 1-dimensional label
# note the difference in size between logits and labels
loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)


# testing the given set of words
def test(words):
    input_batch, target_batch = make_batch(words)
    guess = sess.run(prediction, feed_dict={X: input_batch, Y: target_batch})

    for i, seq in enumerate(words):
        print(seq[0:n_stage], char_arr[guess[i]])
    print()


# session and main flow
sess = tf.Session()
sess.run(tf.global_variables_initializer())

test(seq_data)
input_batch, target_batch = make_batch(seq_data)
for epoch in range(total_epoch):
    _, error = sess.run([optimizer, loss], feed_dict={X: input_batch, Y: target_batch})
    print('epoch: %04d' % epoch, 'error = %.4f' % error)

print()
info()
test(seq_data)
seq_data = ['topi', 'kore', 'japa', 'cros', 'enem']
# seq_data = ['topic', 'korea', 'japan', 'cross', 'enemy'] # if seq[0:len(seq_data[0])-1]

test(seq_data)

/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


(4, ?, 128)
(?, 128)
eage l
enem l
frui l
grac l
laug l
mone l
peac l
part l
read l
stag l

epoch: 0000 error = 3.6324
epoch: 0001 error = 3.0854
epoch: 0002 error = 1.9044
epoch: 0003 error = 6.2722
epoch: 0004 error = 1.3935
epoch: 0005 error = 1.9103
epoch: 0006 error = 1.7403
epoch: 0007 error = 1.4842
epoch: 0008 error = 1.1420
epoch: 0009 error = 0.8397
epoch: 0010 error = 0.6927
epoch: 0011 error = 0.6378
epoch: 0012 error = 0.4560
epoch: 0013 error = 0.4587
epoch: 0014 error = 0.1886
epoch: 0015 error = 0.1454
epoch: 0016 error = 0.1056
epoch: 0017 error = 0.0470
epoch: 0018 error = 0.0121
epoch: 0019 error = 0.5094
epoch: 0020 error = 0.0488
epoch: 0021 error = 0.7401
epoch: 0022 error = 0.0125
epoch: 0023 error = 0.0127
epoch: 0024 error = 0.0034
epoch: 0025 error = 0.0035
epoch: 0026 error = 0.0483
epoch: 0027 error = 0.0003
epoch: 0028 error = 0.0026
epoch: 0029 error = 0.0016
epoch: 0030 error = 0.0005
epoch: 0031 error = 0.0244
epoch: 0032 error = 0.0244
epoch: 0033 error